In [3]:
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

In [ ]:
my_model = Sequential()
my_model.add( Conv2D(
        filters=16, 
        kernel_size=(3, 3), 
        strides=(1,1),
        activation='relu',
   
        input_shape=(256,256,3) 
))
my_model.add(BatchNormalization())
my_model.add(MaxPooling2D())
    
my_model.add(Conv2D( filters=32, kernel_size=(3, 3), activation='relu')) 
my_model.add(BatchNormalization())
my_model.add(MaxPooling2D()) 

my_model.add(Conv2D( filters=64, kernel_size=(3, 3), activation='relu')) 
my_model.add(BatchNormalization())
my_model.add(MaxPooling2D())
    
my_model.add(Flatten())

my_model.add(Dense(512, activation='relu')) 
my_model.add(Dropout(0.09)) 
my_model.add(Dense(1, activation='sigmoid'))
    
my_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

my_model.load_weights('') #place weights file here
print("Model weights loaded successfully.")

<h3> Test Data Set Here:</h3>

In [ ]:
def load_true_labels(directory):
    true_labels = []
    for filename in os.listdir(directory):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            if 'REAL' in directory:
                true_labels.append('Real')
            elif 'FAKE' in directory:
                true_labels.append('Fake')
    return true_labels



def predict_images_from_directory(directory, model, target_size=(256, 256), threshold=0.5, batch_size=512):
    results = []
    images = []
    
    for filename in os.listdir(directory):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(directory, filename)
            img = load_img(img_path, target_size=target_size)
            img = img_to_array(img)
            img = img / 255.0
            images.append(img)

            # When the batch is full, make predictions
            if len(images) == batch_size:
                predictions = model.predict(np.array(images)) #Predicting here
                for prediction in predictions:
                    label = 'Real' if prediction[0] >= threshold else 'Fake'
                    results.append(label)
                images = []  # Reset the batch
    
    # Process any remaining images that didn't fill a complete batch
    if images:
        predictions = model.predict(np.array(images))
        for prediction in predictions:
            label = 'Real' if prediction[0] >= threshold else 'Fake'
            results.append(label)
    
    return results


# Path to your directories
real_directory = ''
fake_directory = ''

# Load true labels
true_labels_real = load_true_labels(real_directory)
true_labels_fake = load_true_labels(fake_directory)
true_labels = true_labels_real + true_labels_fake # Combine labels lists

# Get predictions from both directories
predictions_real = predict_images_from_directory(real_directory, my_model)
predictions_fake = predict_images_from_directory(fake_directory, my_model)
predictions = predictions_real + predictions_fake # Combine predictions lists

# Generate confusion matrix
cm = confusion_matrix(true_labels, predictions, labels=['Real', 'Fake'])

# Calculate accuracy
correct_predictions = np.sum(np.array(true_labels) == np.array(predictions))
total_predictions = len(predictions)
accuracy = correct_predictions / total_predictions * 100

# Print accuracy
print(f'Accuracy: {accuracy:.2f}%')


# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Real', 'Fake'], yticklabels=['Real', 'Fake'])
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix')
plt.show()

<h2>Conduct Analysis Below:</h2>